In [6]:
#Dave's code

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time #Included to use in pausing between API requests
import re  #Enable regular expressions
import os

# Import API key
from api_keys import api_key


In [47]:
#Make API calls to US Census Bureau - American Community Survey 1-Year Data
#https://www.census.gov/data/developers/data-sets/acs-1year.2017.html
#Data will be returned on a county & state basis

#Set the range of years to pull from the api
years = ["2012","2013","2014","2015","2016","2017"]

#List of fields to request from the API
#Would like to build this into a file, with columns for the ID required for the API call and a human readable name for output
fields = ["NAME","S0101_C01_001E","S1701_C01_001E","S1701_C02_001E","S1702_C01_001E","S1702_C02_001E",
          "S1703_C01_001E","S1703_C02_001E"]

#Initiate an empty list to hold census records
census_data_raw = []

#Turn the list of fields into a string to use in the api url
field_list = ','.join(fields)

#Base url for the API request. YEAR and FIELDS are placeholders and will be substituted with values
#Using * for county and state will pull all available records
census_base_url = "https://api.census.gov/data/YEAR/acs/acs1/subject?get=FIELDS&for=county:*&in=state:*&key="

#Since API url has a year component, loop through the list of years to pull back the relevant data
for year in years:
    census_url = re.sub("YEAR", year, census_base_url) #Use regular expression to substitute the year into the base url
    census_url = re.sub("FIELDS", field_list, census_url) #Use regular expression to substitute in list of fields
    #print(census_url) #for debugging
    response = requests.get(census_url + api_key).json()
    #print(response) #for debugging
    for census_record in response[1:]: #Ignore first record in the loop
        census_record.append(year)  #Append the year to the end of the returned record
        census_data_raw.append(census_record)  #Append the full record to the list of records
    time.sleep(1) #Sleep for a second, because APIs




In [ ]:
#Set path for output file
output_data_file = "census_data\census_data.tsv"

#Use the list of fields as the headers for the data frame
headers = fields 
#Add titles for fields returned in API that aren't part of requested fields
headers.extend(["State Code","County Code","Year"])

#Convert list of census data to a dataframe
census_data_raw_df = pd.DataFrame(census_data_raw, columns=headers)

#Remove the words Parish, County, Municipio from names to match up to EPA data set
census_data_raw_df["CountyState Name"] = census_data_raw_df["CountyState Name"].str.replace(' Parish', '')
census_data_raw_df["CountyState Name"] = census_data_raw_df["CountyState Name"].str.replace(' County', '')
census_data_raw_df["CountyState Name"] = census_data_raw_df["CountyState Name"].str.replace(' Municipio', '')

#Separate the county and state into distinct fields, based on comma, remove preceding space on state
census_data_raw_df[['County','State']]=census_data_raw_df['CountyState Name'].str.split(',',expand=True).apply(lambda x: x.str.strip())

#Export dataframe to a tab-delimited file, since one of the columns includes a comma
census_data_raw_df.to_csv(output_data_file, sep="\t", index=False, encoding='utf-8')

#show the dataframe
census_data_raw_df

In [3]:
#show the dataframe

census_data_raw_df = pd.read_csv('census_data\census_data.tsv', sep="\t")
census_data_raw_df

,CountyState Name,Total Population,Total Under 18,Total 18 to 64,Total 65 and over,Total Male,Total Female,Total 16 and Over,Total 16 and Over Employed,InPoverty Population,...,PctBelowPoverty 65 and over,PctBelowPoverty Male,PctBelowPoverty Female,PctBelowPoverty 16 and Over,PctBelowPoverty 16 and Over Employed,State Code,County Code,Year,County,State
0,"Ouachita, Louisiana",146095,39226,67590,53615,-999999999,-999999999,5814,2475,35443,...,40.7,-999999999.0,-999999999.0,48.2,28.2,22,73,2012,Ouachita,Louisiana
1,"Rapides, Louisiana",128128,33709,60855,40335,-999999999,-999999999,5011,2521,28867,...,37.8,-999999999.0,-999999999.0,54.8,38.8,22,79,2012,Rapides,Louisiana
2,"St. Landry, Louisiana",82325,22661,39189,33739,-999999999,-999999999,2047,969,21149,...,43.5,-999999999.0,-999999999.0,62.0,46.2,22,97,2012,St. Landry,Louisiana
3,"St. Tammany, Louisiana",237342,59422,114493,27636,-999999999,-999999999,10713,6239,32212,...,28.4,-999999999.0,-999999999.0,33.1,34.8,22,103,2012,St. Tammany,Louisiana
4,"Tangipahoa, Louisiana",119480,30126,58332,36205,-999999999,-999999999,6504,2857,29110,...,38.2,-999999999.0,-999999999.0,38.9,63.8,22,105,2012,Tangipahoa,Louisiana
5,"Terrebonne, Louisiana",110088,28085,53178,19656,-999999999,-999999999,3278,1464,20330,...,38.5,-999999999.0,-999999999.0,39.0,33.4,22,109,2012,Terrebonne,Louisiana
6,"Androscoggin, Maine",104067,22796,51042,-999999999,-999999999,-999999999,4835,2876,16593,...,-999999999.0,-999999999.0,-999999999.0,34.8,35.9,23,1,2012,Androscoggin,Maine
7,"Aroostook, Maine",68715,13087,33689,-999999999,-999999999,-999999999,2818,1603,11358,...,-999999999.0,-999999999.0,-999999999.0,38.0,33.7,23,3,2012,Aroostook,Maine
8,"Cumberland, Maine",275849,56165,134230,6048,-999999999,6187,9754,5588,32297,...,29.1,-999999999.0,28.9,26.7,25.6,23,5,2012,Cumberland,Maine
9,"Kennebec, Maine",118408,23831,57188,-999999999,-999999999,-999999999,5885,3085,17804,...,-999999999.0,-999999999.0,-999999999.0,35.3,34.6,23,11,2012,Kennebec,Maine


In [47]:
# Loading EPA data from our CSV's and create a dataframe of EPA data across all years

import os

def read_all_epa_years ():
    
    # Base file name
    base_name = "annual_aqi_by_county_"
    # epa_data_directory
    data_directory = "epa_data"
    
    years = [2012, 2013, 2014, 2015, 2016, 2017]
    df_list = []
    
    # Loop over the years, read in .csv file and append to list of dataframes
    for year in years:
        file_name = f"{base_name}{year}.csv"
        file_path = os.path.join("", data_directory, file_name)
        
        df_list.append(pd.read_csv(file_path))
    
    # concatenate the years
    all_years_df = pd.concat(df_list, ignore_index=True)
    
    # write the combined dataframe to a new .csv file
    output_path = os.path.join("", data_directory, "annual_aqi_by_county_all.csv")
    all_years_df.to_csv(output_path)
    
    # return the dataframe
    return all_years_df

all_years_df = read_all_epa_years()


In [7]:
# now we merge both of these dataframes together
# census data path
census_data_path = "census_data"
census_file = "census_data.tsv"
census_file_name = os.path.join("", census_data_path, census_file)

census_df = pd.read_csv(census_file_name, sep="\t", encoding='utf-8', na_values=-999999999)

In [ ]:
# epa data path
epa_data_path = "epa_data"
epa_file = "annual_aqi_by_county_all.csv"
epa_file_name = os.path.join("", epa_data_path, epa_file)

epa_df = pd.read_csv(epa_file_name)

In [ ]:
# combining the two dataframes on State, County, and Year
combined_df = pd.merge(epa_df, census_df, on=["State", "County", "Year"])

In [ ]:
# write out the combined data frame
combined_df.to_csv("census_and_epa.csv")

In [18]:
# read the dataframe from csv and take a peek
combined_df = pd.read_csv('census_and_epa.csv')

# some columns, most likely defunct indexes, survived the merge and needed to be deleted
combined_df = combined_df.drop(combined_df.columns[[0, 1]], axis=1)
combined_df.head()

,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,...,PctBelowPoverty Population,PctBelowPoverty Under 18,PctBelowPoverty 18 to 64,PctBelowPoverty 65 and over,PctBelowPoverty Male,PctBelowPoverty Female,PctBelowPoverty 16 and Over,PctBelowPoverty 16 and Over Employed,State Code,County Code
0,Alabama,Baldwin,2012,284,226,56,2,0,0,0,...,14.1,20.2,14.2,30.5,NaN,NaN,20.1,22.8,1,3
1,Alabama,DeKalb,2012,361,282,74,5,0,0,0,...,13.3,20.4,12.9,NaN,NaN,NaN,36.6,31.7,1,49
2,Alabama,Elmore,2012,245,212,33,0,0,0,0,...,13.0,19.9,10.1,19.0,NaN,NaN,14.5,11.8,1,51
3,Alabama,Etowah,2012,277,229,48,0,0,0,0,...,21.2,32.4,19.8,27.8,NaN,NaN,40.5,41.0,1,55
4,Alabama,Houston,2012,284,238,46,0,0,0,0,...,19.2,28.8,17.1,38.8,NaN,NaN,38.4,31.9,1,69


In [39]:
# show columns to make sure everything aligns properly and provide a summary of our final dataframe

print(f'our final dataframe contains {combined_df.shape[0]} rows and {combined_df.shape[1]} columns')
county_count = combined_df['County'].nunique()
state_count = combined_df['State'].nunique()
year_count = combined_df['Year'].nunique()
print(f'\nour final dataframe contains data from {county_count} counties across {state_count} states/territories spanning {year_count} years')
print(f'\nour final dataframe has the following column names: \n {combined_df.columns}')

our final dataframe contains 3659 rows and 46 columns

our final dataframe contains data from 522 counties across 50 states/territories spanning 6 years

our final dataframe has the following column names: 
 Index(['State', 'County', 'Year', 'Days with AQI', 'Good Days',
       'Moderate Days', 'Unhealthy for Sensitive Groups Days',
       'Unhealthy Days', 'Very Unhealthy Days', 'Hazardous Days', 'Max AQI',
       '90th Percentile AQI', 'Median AQI', 'Days CO', 'Days NO2',
       'Days Ozone', 'Days SO2', 'Days PM2.5', 'Days PM10', 'CountyState Name',
       'Total Population', 'Total Under 18', 'Total 18 to 64',
       'Total 65 and over', 'Total Male', 'Total Female', 'Total 16 and Over',
       'Total 16 and Over Employed', 'InPoverty Population',
       'InPoverty Under 18', 'InPoverty 18 to 64', 'InPoverty 65 and Over',
       'InPoverty Male', 'InPoverty Female', 'InPoverty 16 and Over',
       'InPoverty 16 and Over Employed', 'PctBelowPoverty Population',
       'PctBelowPover